In [ ]:
import six

import SimpleITK as sitk
import numpy as np
import pandas as pd
from radiomics import featureextractor
import matplotlib.pyplot as plt

def calculate_features(scan, row_dict = {}):
    extractor = featureextractor.RadiomicsFeatureExtractor("config.yml") 
    mask = np.ones_like(scan)
    mask[0,0] = 0
    result = extractor.execute(sitk.GetImageFromArray(scan), sitk.GetImageFromArray(mask) ,voxelBased=False)
    extractor.enableAllImageTypes()
    names = []
    vals = []
    for k,v in row_dict.items():
        names.append(k)
        vals.append(v)
    for key, val in six.iteritems(result):
        if 'diagn' not in key:
            names.append(key)
            vals.append(val)
    return pd.Series(data=vals, index = names)


In [ ]:
print("Starting")

In [ ]:
import os
import pandas as pd
import numpy as np
import pandas as pd

def split_into_tiles(image, tile_size=30):
    w, h = 512, 1024
    # Adjusting the height and width to be divisible by tile_size
    h = (h // tile_size) * tile_size
    w = (w // tile_size) * tile_size
    
    tiles = np.zeros((h // tile_size, w // tile_size), dtype=int)  # Ensuring binary tiles
    
    for i in range(0, h, tile_size):
        for j in range(0, w, tile_size):
            # Take the tile and check if there is any '1' in the mask (binary representation)
            tile = image[i:i+tile_size, j:j+tile_size]
            tiles[i // tile_size, j // tile_size] = np.any(tile)  # Mark tile as 1 if there's any non-zero pixel
    
    return tiles

def calculate_iou(df):
    grouped = df.groupby(["model_name", "npz_name"])
    results = []

    for (model_name, npz_name), group in grouped:
        clear_image_mask = None
        other_masks = []

        # Collect masks for this group
        for _, row in group.iterrows():
            data = np.load(row["full_path"], allow_pickle=True)
            if "lime" in data:
                transformed_mask = split_into_tiles(data["lime"])
                if row["mode"] == "clear_images":
                    clear_image_mask = transformed_mask
                else:
                    other_masks.append((row["mode"], transformed_mask))

        # Initialize dictionary for storing IoUs
        iou_dict = {}

        # Calculate IoU for each non-"clear_image" mask with the "clear_image" mask
        if clear_image_mask is not None:
            for mode, mask in other_masks:
                intersection = np.logical_and(clear_image_mask, mask).sum()
                union = np.logical_or(clear_image_mask, mask).sum()
                iou = intersection / union if union > 0 else 0
                iou_dict[f"iou_{mode}"] = iou
        else:
            iou_dict["error"] = "No clear_image mask found for this group"

        # Append result for the current group
        result_entry = {
            "model_name": model_name,
            "npz_name": npz_name
        }
        result_entry.update(iou_dict)
        results.append(result_entry)

    return pd.DataFrame(results)


In [ ]:
import os
import pandas as pd

def extract_npz_paths(root_dir):
    data = []
    
    # Collect data from the directory
    for dirpath, _, filenames in os.walk(root_dir):
        for file in filenames:
            if file.endswith(".npz"):
                full_path = os.path.join(dirpath, file)
                parts = full_path.split(os.sep)
                
                if len(parts) >= 3:  # Ensure there are enough parts
                    entry = {
                        "full_path": full_path,
                        "model_name": parts[-3],
                        "mode": parts[-2],
                        "npz_name": parts[-1]
                    }
                    data.append(entry)

    # Convert to DataFrame
    df = pd.DataFrame(data)
    
    # Group by 'model_name' and 'mode' and select the first 10 samples from each group
    df_grouped = df.groupby(['model_name', 'mode']).head(100).reset_index(drop=True)
    
    # Unwinding (resetting index ensures it's fully flattened)
    df_flat = df_grouped.reset_index(drop=True)

    return df_flat


In [ ]:
root_dir_df = extract_npz_paths("./explanation3")

In [ ]:
root_dir_df.to_csv("paths.csv", index=False)

### Calculating statistics

In [ ]:
import argparse
import six

import SimpleITK as sitk
import numpy as np
import pandas as pd
from radiomics import featureextractor
import matplotlib.pyplot as plt

def calculate_features(scan, row_dict = {}):
    extractor = featureextractor.RadiomicsFeatureExtractor("config.yml") 
    mask = np.ones_like(scan)
    mask[0,0] = 0
    result = extractor.execute(sitk.GetImageFromArray(scan), sitk.GetImageFromArray(mask) ,voxelBased=False)
    extractor.enableAllImageTypes()
    names = []
    vals = []
    for k,v in row_dict.items():
        names.append(k)
        vals.append(v)
    for key, val in six.iteritems(result):
        if 'diagn' not in key:
            names.append(key)
            vals.append(val)
    return pd.Series(data=vals, index = names)



if __name__ == "__main__":
    args = {
        "from": 0,
        "to": len(root_dir_df)
    }
    df = pd.read_csv("./paths.csv")
    print(len(df))
    result = []
    for idx, row in df.iloc[args['from']:args['to']].iterrows():
        print(idx)
        row_dict = dict(row)
        array = np.load(row["full_path"], allow_pickle=True)
        lime = (array['lime'] + 1) * 127.5
        lime = array['lime']
        features = calculate_features(lime, row_dict)
        result.append(features)
        pd.DataFrame(result).to_csv(f"{args['from']}-{args['to']}_no_scaling.csv")

    


### Calculating IoU statistics

In [ ]:
root_dir_df.groupby(['model_name', 'mode']).first()

In [ ]:
iou_df = calculate_iou(root_dir_df)

In [ ]:
iou_df['avg_iou'] = iou_df[['iou_full', 'iou_no_header', 'iou_text', 'iou_wrinkles']].mean(axis=1)

# Sort the DataFrame by 'avg_iou' in descending order
sorted_iou_scores_df = iou_df.sort_values(by='avg_iou', ascending=False)

# Display the result
sorted_iou_scores_df[['npz_name', 'model_name', 'avg_iou']].to_csv("test111.csv")

In [ ]:
sorted_iou_scores_df.head(2)

In [ ]:
iou_df.drop(columns=['npz_name']).groupby(['model_name']).mean().round(3)

In [ ]:
import re

# Replace model names with specified new names
iou_df['model_name'] = iou_df['model_name'].replace({
    'phd-dk_ptb_xl_training_xai_fixed_model-htdatole_v0': "Model 2",
    'phd-dk_ptb_xl_training_xai_fixed_model-jr52wzjb_v0': "Model 5",
    'phd-dk_ptb_xl_training_xai_fixed_model-z2gk30z8_v0': "Model 1"
})

# Extract number from 'npz_name' and remove leading zeros
def extract_number_from_npz_name(name):
    match = re.search(r'(\d+)_', name)
    if match:
        return str(int(match.group(1)))  # Convert to int to remove leading zeros
    return name  # Fallback to the original name if no match

iou_df['npz_name'] = iou_df['npz_name'].apply(extract_number_from_npz_name)

# Selector for method: Choose between 'best_worst' and 'percentiles'
method = 'percentiles'  # Change to 'best_worst' or 'percentiles'

result_dfs = []

for model in iou_df['model_name'].unique():
    # Filter for the current model
    filtered_df = iou_df[iou_df['model_name'] == model]
    
    if method == 'best_worst':
        # Sort by the metric (replace 'avg_iou' with your actual metric column)
        sorted_df = filtered_df.sort_values(by='avg_iou', ascending=False)
        
        # Select top 3 and bottom 3
        best_examples = sorted_df.head(3)
        worst_examples = sorted_df.tail(3)
        
        # Combine best and worst examples
        combined_df = pd.concat([best_examples, worst_examples])

    elif method == 'percentiles':
        # Calculate the 75th and 25th percentiles
        p75 = filtered_df['avg_iou'].quantile(0.75)
        p25 = filtered_df['avg_iou'].quantile(0.25)
        
        # Select examples closest to 75th and 25th percentiles
        p75_examples = filtered_df.iloc[(filtered_df['avg_iou'] - p75).abs().argsort()[:1]]
        p25_examples = filtered_df.iloc[(filtered_df['avg_iou'] - p25).abs().argsort()[:1]]
        
        # Combine percentile examples
        combined_df = pd.concat([p75_examples, p25_examples])

    # Add to list of results
    result_dfs.append(combined_df)

# Combine all results for each model into one DataFrame
final_df = pd.concat(result_dfs)

# Round all numeric columns to 3 significant digits
def round_to_sig_digits(x, sig_digits=4):
    if isinstance(x, (int, float)):
        return f"{x:.{sig_digits-1}g}"  # Format to significant digits
    return x

final_df = final_df.applymap(round_to_sig_digits)

# Save the combined results to LaTeX
final_df.to_latex("median_examples_all_models3.lx", index=False)


### Calculating pyradiomic statistics

In [ ]:
import re

# Specify the directory to search
directory = "."

# Define the pattern (e.g., XX-YY.csv)
pattern =  re.compile(r'^\d+-\d+_no_scaling\.csv$')
paths = []
# Walk through the directory and match files
for root, _, files in os.walk(directory):
    for file in files:
        if pattern.match(file) and "old" not in os.path.join(root, file):
            paths.append(os.path.join(root, file))

In [ ]:
dataframes = [pd.read_csv(path) for path in paths]

In [ ]:
df =  pd.concat(dataframes, ignore_index=True)
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
from dython.nominal import associations

In [ ]:
df.index = df.full_path
feature_columns = df.select_dtypes(include=['number']).columns
features = df[feature_columns]


In [ ]:
len(feature_columns)

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score

# Step 1: Standardize the features (mean=0, std=1)
scaler = StandardScaler()
standardized_features = scaler.fit_transform(features)

# Step 2: Apply PCA to reduce dimensionality while retaining 95% of the variance
pca = PCA(n_components=0.99)
pca_features = pca.fit_transform(standardized_features)

# Step 3: Create a DataFrame for the PCA-transformed data
pca_df = pd.DataFrame(pca_features, columns=[f"PC{i+1}" for i in range(pca_features.shape[1])])
pca_df.index = df.full_path

# Step 4: Optionally, merge the PCA results back with the original data
combined_with_pca = pd.concat([df, pca_df], axis=1)

# Step 5: Find the optimal number of clusters using the Elbow Method and Silhouette Score
def find_optimal_k(data, mode, max_clusters=10):
    wcss = []  # Within-cluster sum of squares
    silhouette_scores = []

    for k in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        clusters = kmeans.fit_predict(data)
        wcss.append(kmeans.inertia_)

        # Calculate silhouette score
        score = silhouette_score(data, mode)
        silhouette_scores.append(score)

    # Print the optimal k based on the highest silhouette score
    print(silhouette_scores)
    optimal_k = silhouette_scores.index(max(silhouette_scores)) + 2
    print(f"Optimal number of clusters (based on Silhouette Score): {optimal_k}")

    return optimal_k

# Step 6: Find the optimal number of clusters
optimal_k = find_optimal_k(pca_features, combined_with_pca['model_name'],max_clusters=10)

# Step 7: Apply KMeans with the optimal number of clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
clusters = kmeans.fit_predict(pca_features)
combined_with_pca['cluster'] = clusters

# Step 8: Optional - Perform associations or further analysis on the clusters
r = associations(combined_with_pca[['mode', 'cluster']])


#### Veryfing the most important pca features

In [ ]:
n_pcs= pca.components_.shape[0]

# get the index of the most important feature on EACH component
# LIST COMPREHENSION HERE
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
most_important_names = [features.columns[most_important[i]] for i in range(n_pcs)]
most_important_names

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
def accuracy_per_class(y_true, y_pred):
    """
    Calculate classification accuracy per class.
    
    Parameters:
    -----------
    y_true : array-like
        Ground truth labels
    y_pred : array-like
        Predicted labels
    
    Returns:
    --------
    dict : Dictionary mapping class labels to their respective accuracy scores
    """
    # Create confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Get the list of unique classes
    classes = np.unique(np.concatenate([y_true, y_pred]))
    
    # Calculate accuracy for each class
    accuracies = {}
    for i, class_label in enumerate(classes):
        # True positives for this class
        tp = cm[i, i]
        
        # Total samples of this class in ground truth
        total = np.sum(cm[i, :])
        
        # Calculate accuracy
        accuracy = tp / total if total > 0 else 0
        accuracies[class_label] = accuracy
    
    return accuracies

#### Training simple classifier to perform model analysis

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Step 1: Standardize the features
scaler = StandardScaler()
standardized_features = scaler.fit_transform(features)

# Step 2: Apply PCA
pca = PCA(n_components=0.99)
pca_features = pca.fit_transform(standardized_features)

# Step 3: Create a DataFrame for the PCA-transformed data
pca_df = pd.DataFrame(pca_features, columns=[f"PC{i+1}" for i in range(pca_features.shape[1])])
pca_df.index = df.full_path

# Merge the PCA results with the original DataFrame
combined_with_pca = pd.concat([df, pca_df], axis=1)

# Step 4: Add KMeans clustering
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(pca_features)
combined_with_pca['cluster'] = clusters

# Step 5: Prepare data for classification
X = pca_df  # PCA-transformed features
combined_with_pca['mode'] = combined_with_pca['mode'].replace({'clear_images': "Base Image",
 "full": "Discoloration",
 "no_header": "No patient Data",
 "text": "Handwriting",
 "wrinkles": "Wrinkles"})
y = combined_with_pca['mode']  # Target variable

# Step 6: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 7: Train a classification model
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Step 8: Make predictions
y_pred = clf.predict(X_test)

# Step 9: Evaluate the classification model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Step 10: Print evaluation metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

cm = confusion_matrix(y_test, y_pred)

# Extract class names (ensures labels match correctly)
class_names = y.unique()  # Extract unique class labels for display

# Convert confusion matrix to a DataFrame for better readability
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)

# Print the confusion matrix as a table
print("Confusion Matrix:")
cm_df
print(accuracy_per_class(y_test, y_pred))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Step 1: Standardize the features
scaler = StandardScaler()
standardized_features = scaler.fit_transform(features)

# Step 2: Apply PCA
pca = PCA(n_components=0.99)
pca_features = pca.fit_transform(standardized_features)

# Step 3: Create a DataFrame for the PCA-transformed data
pca_df = pd.DataFrame(pca_features, columns=[f"PC{i+1}" for i in range(pca_features.shape[1])])
pca_df.index = df.full_path

# Merge the PCA results with the original DataFrame
combined_with_pca = pd.concat([df, pca_df], axis=1)

# Step 4: Add KMeans clustering
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(pca_features)
combined_with_pca['cluster'] = clusters

# Step 5: Prepare data for classification
X = pca_df  # PCA-transformed features
combined_with_pca['model_name'] = combined_with_pca['model_name'].replace({'phd-dk_ptb_xl_training_xai_fixed_model-htdatole_v0': "Model 2",
 "phd-dk_ptb_xl_training_xai_fixed_model-jr52wzjb_v0": "Model 5",
 "phd-dk_ptb_xl_training_xai_fixed_model-z2gk30z8_v0": "Model 1"
})
y = combined_with_pca['model_name']  # Target variable

# Step 6: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 7: Train a classification model
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Step 8: Make predictions
y_pred = clf.predict(X_test)

# Step 9: Evaluate the classification model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Step 10: Print evaluation metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

cm = confusion_matrix(y_test, y_pred)

# Extract class names (ensures labels match correctly)
class_names = y.unique()  # Extract unique class labels for display

# Convert confusion matrix to a DataFrame for better readability
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)

# Print the confusion matrix as a table
print("Confusion Matrix:")
cm_df
print(accuracy_per_class(y_test, y_pred))

In [ ]:
len(pca_features[0])

In [ ]:
pca_features.shape[1]

In [ ]:
explained_variance = pca.explained_variance_ratio_

# Step 4: Get the principal components
components = pca.components_

# Step 5: Display the most important features for each principal component
# Here we take the absolute values to see the most influential features
importance = pd.DataFrame(components, columns=feature_columns)


In [ ]:
pca.explained_variance_

In [ ]:
importance = pd.DataFrame(components, columns=feature_columns)

# Sum the absolute values across all components to get the total importance
total_importance = importance.abs().sum(axis=0) / len(importance)

# Step 5: Sort the features by their total importance
sorted_importance = total_importance.sort_values(ascending=False)

# Display the most important features
print(sorted_importance.head(8))

In [ ]:
pca.components_

#### Performing clusteristion of model used and method of augmentation

In [ ]:
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams["font.size"] = 18

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Standardize the features (mean=0, std=1)
scaler = StandardScaler()
standardized_features = scaler.fit_transform(features)

# Step 2: Apply PCA
pca = PCA(n_components=0.99)
pca_features = pca.fit_transform(standardized_features)

# Step 3: Create a DataFrame for the PCA-transformed data
pca_df = pd.DataFrame(pca_features, columns=[f"PC{i+1}" for i in range(pca_features.shape[1])])
pca_df.index = df.full_path

# Optionally, you can merge the PCA results back to the original DataFrame
combined_with_pca = pd.concat([df, pca_df], axis=1)
tsne = TSNE(n_components=2, random_state=42)
tsne_features = tsne.fit_transform(pca_features)  # Apply t-SNE on PCA features

# Add the t-SNE results to the DataFrame
combined_with_pca['tSNE-1'] = tsne_features[:, 0]
combined_with_pca['tSNE-2'] = tsne_features[:, 1]

# Step 4: Apply KMeans clustering
kmeans = KMeans(n_clusters=6, random_state=42)
clusters = kmeans.fit_predict(tsne_features)
combined_with_pca['cluster'] = clusters

# Step 5: Apply t-SNE for 2D visualization
combined_with_pca['model_name'] = combined_with_pca['model_name'].replace({'phd-dk_ptb_xl_training_xai_fixed_model-htdatole_v0': "Model 2",
 "phd-dk_ptb_xl_training_xai_fixed_model-jr52wzjb_v0": "Model 5",
 "phd-dk_ptb_xl_training_xai_fixed_model-z2gk30z8_v0": "Model 1"
})
# Step 6: Visualize the clusters using a scatter plot
plt.figure(figsize=(8, 8))
sns.scatterplot(x='tSNE-1', y='tSNE-2', hue='model_name', palette='Set1', data=combined_with_pca, s=100, marker='o', legend='full')

# Customize plot
plt.xlabel("t-SNE component 1")
plt.ylabel("t-SNE component 2")
plt.legend(title="Model", loc='upper left')
# plt.show()
plt.tight_layout()
plt.savefig("model_name.pdf", dpi=1000, format="pdf")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Standardize the features (mean=0, std=1)
scaler = StandardScaler()
standardized_features = scaler.fit_transform(features)

# Step 2: Apply PCA
pca = PCA(n_components=0.99)
pca_features = pca.fit_transform(standardized_features)

# Step 3: Create a DataFrame for the PCA-transformed data
pca_df = pd.DataFrame(pca_features, columns=[f"PC{i+1}" for i in range(pca_features.shape[1])])
pca_df.index = df.full_path

# Optionally, you can merge the PCA results back to the original DataFrame
combined_with_pca = pd.concat([df, pca_df], axis=1)
tsne = TSNE(n_components=2, random_state=42)
tsne_features = tsne.fit_transform(pca_features)  # Apply t-SNE on PCA features

# Add the t-SNE results to the DataFrame
combined_with_pca['tSNE-1'] = tsne_features[:, 0]
combined_with_pca['tSNE-2'] = tsne_features[:, 1]

# Step 4: Apply KMeans clustering
kmeans = KMeans(n_clusters=6, random_state=42)
clusters = kmeans.fit_predict(tsne_features)
combined_with_pca['cluster'] = clusters

# Step 5: Apply t-SNE for 2D visualization

combined_with_pca['mode'] = combined_with_pca['mode'].replace({'clear_images': "Base Image",
 "full": "Discoloration",
 "no_header": "No patient Data",
 "text": "Handwriting",
 "wrinkles": "Wrinkles"})
# Step 6: Visualize the clusters using a scatter plot
plt.figure(figsize=(8, 8))
sns.scatterplot(x='tSNE-1', y='tSNE-2', hue='mode', palette='Set1', data=combined_with_pca, s=100, marker='o', legend='full')

# Customize plot
plt.xlabel("t-SNE component 1")
plt.ylabel("t-SNE component 2")
plt.legend(title="Modifications", loc='upper left')
# plt.show()
plt.tight_layout()
plt.savefig("mode.pdf", dpi=1000, format="pdf")